## Audio

In [ ]:
import tensorflow as tf
import sounddevice as sd
import scipy.io.wavfile as wav
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch

In [2]:

gpus = tf.config.list_physical_devices('GPU')
print("GPUs Available:", len(gpus))
for gpu in gpus:
    print(gpu)

print(torch.cuda.is_available())
print(torch.cuda.device_count())


GPUs Available: 1
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
True
1


W0000 00:00:1749742862.405234   48337 gpu_device.cc:2430] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


In [ ]:
# Record audio from microphone
def record_audio(duration=5, fs=16000):
    print("Recording...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    print("Recording finished.")
    return audio.flatten(), fs


# Save audio to a temporary WAV file
def save_wav(audio, fs, filename="temp.wav"):
    wav.write(filename, fs, audio)
    return filename


def create_pipeline(only_cpu: bool = False) -> pipeline:
    device = "cuda:0" if not only_cpu and torch.cuda.is_available() else "cpu"
    print(f"Using device {device}.")
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    model_id = "openai/whisper-medium"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)
    return pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )


pipe = create_pipeline(only_cpu=False)


# Load and transcribe audio
def transcribe(filename):
    result = pipe(filename)
    print("Transcription:", result)

In [ ]:
audio, fs = record_audio(duration=10, fs=16000)
filename = save_wav(audio, fs)
transcribe(filename)

In [ ]:
# import sounddevice as sd
# import numpy as np
#
# def audio_stream(chunk_duration=1, fs=16000):
#     chunk_samples = int(chunk_duration * fs)
#     with sd.InputStream(samplerate=fs, channels=1, dtype='int16') as stream:
#         print("Live transcription started. Speak into the microphone.")
#         while True:
#             audio_chunk, _ = stream.read(chunk_samples)
#             yield audio_chunk.flatten()
#
# for chunk in audio_stream():
#     audio_float = (chunk / 32768.0).astype(np.float32)
#     # Pass as dict with 'array' and 'sampling_rate'
#     result = pipe({"array": audio_float, "sampling_rate": 16000})
#     print("You said:", result["text"])